# Multiplicity determination with Calibr

Note: Uses the R3BRoot NeuLAND module. As this module is compiled with ROOT, it is available in Python by `import ROOT` if the environment variables are set.

In [1]:
import numpy as np
import pandas as pd
import os
import ROOT

Welcome to JupyROOT 6.16/00


## Example Fit / Calibration
We can run the full calibraion from python, with some amount of yucky syntax

In [2]:
def calibrate_calibr(infiles, outfile, max_neutrons, image):
    cal = ROOT.Neuland.Neutron2DCalibr(max_neutrons)
    for file in infiles:
        print(f"Processing {file} ...", flush=True)
        cal.AddClusterFile(file)
    vslope = ROOT.std.vector("double")()
    vslope += [0.04, 0.001, 0.001, 10]
    vdistance = ROOT.std.vector("double")()
    vdistance += [10, 0.5, 6, 60]
    vdist_off = ROOT.std.vector("double")()
    vdist_off += [3, 0.5, 3, 6]
    cal.Optimize(vslope, vdistance, vdist_off)
    cal.WriteParameterFile(outfile)
    cal.Draw(image)

In [ ]:
max_neutrons = 5
num_dp = 30

infiles = [
    f"simulation/training_600AMeV_{num_dp}dp_{neutrons}n.digi.root"
    for neutrons in range(1, max_neutrons + 1)
]
outfile = f"data/600AMeV_{num_dp}dp_{max_neutrons}n.ncut.root"
image = f"data/600AMeV_{num_dp}dp_{max_neutrons}n.png"
os.remove(outfile)
calibrate_calibr(infiles, outfile, max_neutrons, image)

Processing simulation/training_600AMeV_30dp_1n.digi.root ...
Processing simulation/training_600AMeV_30dp_2n.digi.root ...


## Example Prediction

In [ ]:
file = ROOT.TFile.Open(f"data/600AMeV_{num_dp}dp_{max_neutrons}n.ncut.root")
para = file.Get("R3BNeulandNeutron2DPar")

testdata = pd.read_pickle(f"data/600AMeV_{num_dp}dp.trifeature.pkl").sample(frac=1)
y_true = testdata[["nPH"]].values.ravel()

y_pred = np.array(
    [
        para.GetNeutronMultiplicity(row["Edep"], row["nClus"])
        for _, row in testdata.iterrows()
    ]
)

print(list(zip(y_true, y_pred))[0:50])

In [ ]:
from sklearn.metrics import (
    balanced_accuracy_score,
    confusion_matrix,
    plot_confusion_matrix,
)

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

bac = balanced_accuracy_score(y_true, y_pred)
print(bac)
cm = confusion_matrix(y_true, y_pred, labels=range(0, 6))
print(cm)
cmrel = confusion_matrix(y_true, y_pred, labels=range(0, 6), normalize="true")
print(cmrel)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 7))
ax = sns.heatmap(cmrel, cmap="viridis", annot=True)
ax.set_xlabel("Predicted")
ax.set_ylabel("True");